git remote remove origin

git remote add origin git@github.com:EnlightenRK/MEPFIEP.git

In [1]:
import pandas as pd
        
加州工业电价和天然气价格 = pd.read_csv('../Data/AdvanceGuidanceData/加州工业电价和天然气价格.csv', sep = ',')
data =  加州工业电价和天然气价格
data

,time,electricity-price,LNG-value
0,2001-01-01,5.75,7.75
1,2001-02-01,7.70,9.34
2,2001-03-01,7.70,9.84
3,2001-04-01,7.58,10.06
4,2001-05-01,8.05,7.53
...,...,...,...
278,2024-03-01,18.96,13.19
279,2024-04-01,19.30,11.30
280,2024-05-01,21.23,9.11
281,2024-06-01,22.54,9.07


### 平稳性检验：首先，对每个时间序列进行平稳性检验，如ADF（Augmented Dickey-Fuller）检验，确保数据平稳或进行差分处理以达到平稳性。


In [2]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test on both series to check for stationarity
adf_electricity = adfuller(data['electricity-price'])
adf_lng = adfuller(data['LNG-value'])

# Collect results
adf_results = {
    'Electricity Price ADF Statistic': adf_electricity[0],
    'Electricity Price p-value': adf_electricity[1],
    'LNG Value ADF Statistic': adf_lng[0],
    'LNG Value p-value': adf_lng[1]
}
                
adf_results


{'Electricity Price ADF Statistic': 4.325569338834163,
 'Electricity Price p-value': 1.0,
 'LNG Value ADF Statistic': -2.2737837416715263,
 'LNG Value p-value': 0.18056267209364074}

>Electricity Price：ADF统计量为4.33，p值为1.0，说明电价序列非平稳。

>LNG Value：ADF统计量为-2.27，p值为0.18，说明LNG价格序列也非平稳

In [3]:
# Perform first-order differencing to make the series stationary
data['electricity-price-diff'] = data['electricity-price'].diff().dropna()
data['LNG-value-diff'] = data['LNG-value'].diff().dropna()

# Re-run the ADF test on the differenced series
adf_electricity_diff = adfuller(data['electricity-price-diff'].dropna())
adf_lng_diff = adfuller(data['LNG-value-diff'].dropna())

# Collect new results
adf_diff_results = {
    'Differenced Electricity Price ADF Statistic': adf_electricity_diff[0],
    'Differenced Electricity Price p-value': adf_electricity_diff[1],
    'Differenced LNG Value ADF Statistic': adf_lng_diff[0],
    'Differenced LNG Value p-value': adf_lng_diff[1]
}

adf_diff_results


{'Differenced Electricity Price ADF Statistic': -2.169742399889196,
 'Differenced Electricity Price p-value': 0.2174098832044059,
 'Differenced LNG Value ADF Statistic': -5.225113762377162,
 'Differenced LNG Value p-value': 7.811334992544655e-06}

## 差分后的平稳性检验结果
>Differenced Electricity Price：ADF统计量为-2.17，p值为0.217，仍未达到平稳性标准。

>Differenced LNG Value：ADF统计量为-5.23，p值接近0，说明LNG价格序列在一阶差分后平稳。

In [4]:
# Perform second-order differencing on the electricity price series
data['electricity-price-diff2'] = data['electricity-price'].diff().diff().dropna()

# Re-run the ADF test on the second-order differenced series
adf_electricity_diff2 = adfuller(data['electricity-price-diff2'].dropna())

# Collect the new result for the second-order differenced series
adf_diff2_result = {
    'Second-Order Differenced Electricity Price ADF Statistic': adf_electricity_diff2[0],
    'Second-Order Differenced Electricity Price p-value': adf_electricity_diff2[1]
}

adf_diff2_result


{'Second-Order Differenced Electricity Price ADF Statistic': -10.004827713521449,
 'Second-Order Differenced Electricity Price p-value': 1.8432737962005207e-17}

Second-OrderElectricity Price：ADF统计量为-10.00，p值接近0，说明电价序列在二阶差分后达到了平稳性。

In [5]:
from statsmodels.tsa.api import VAR

# Prepare the data for the VAR model using the stationary series
var_data = data[['electricity-price-diff2', 'LNG-value-diff']].dropna()

# Fit the VAR model and select the optimal lag order based on AIC
model = VAR(var_data)
lag_order_results = model.select_order(maxlags=10)  # Checking up to 10 lags

# Display the selected lag order based on AIC
optimal_lag_order = lag_order_results.aic

optimal_lag_order


D:\anaconda\envs\MEPFIEP\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


10

In [6]:
# Fit the VAR model with the selected lag order
var_model = model.fit(optimal_lag_order)

# Summary of the fitted VAR model
var_model_summary = var_model.summary()

var_model_summary


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 06, Nov, 2024
Time:                     21:39:34
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                  -0.387487
Nobs:                     271.000    HQIC:                 -0.721600
Log likelihood:          -598.916    FPE:                   0.388631
AIC:                    -0.945749    Det(Omega_mle):        0.334742
--------------------------------------------------------------------
Results for equation electricity-price-diff2
                                 coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------------------
const                               0.018650         0.043447            0.429           0.668
L1.electricity-price-diff2         -0.771133         0.061048          -12.632   

In [7]:
var_model